In [1]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [3]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [4]:
visits_cart = pd.merge(visits, cart, how = 'left')

Step 3: How long is `visits_cart`?

In [5]:
print(visits_cart.user_id.count())
print(len(visits_cart))
visit_count = len(visits_cart)

2000
2000


Step 4: How many timestamps are null for `cart_time`?

In [6]:
no_cart_count = visits_cart.cart_time.isnull().sum()
print(no_cart_count)
 


1652


Step 5: What percentage only visited?

In [7]:
percent_visited = no_cart_count / visit_count * 100
print(percent_visited)

82.6


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [8]:
cart_checkout = cart.merge(checkout, how = 'left')

cart_no_checkout = cart_checkout[(~cart_checkout.cart_time.isnull()) & (cart_checkout.checkout_time.isnull())]
percent_cart_no_checkout = round((len(cart_no_checkout) / len(cart)) * 100, 1)
print(percent_cart_no_checkout)

35.1


Step 7: Merge it all together

In [9]:
all_data = visits_cart.merge(checkout, how = 'left').merge(purchase, how = 'left')
display(all_data)

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT
...,...,...,...,...,...
2103,33913ac2-03da-45ae-8fc3-fea39df827c6,2017-03-25 03:29:00,NaT,NaT,NaT
2104,4f850132-b99d-4623-80e6-6e61d003577e,2017-01-08 09:57:00,NaT,NaT,NaT
2105,f0830b9b-1f5c-4e74-b63d-3f847cc6ce70,2017-09-07 12:56:00,NaT,NaT,NaT
2106,b01bffa7-63ba-4cd3-9d93-eb1477c23831,2017-07-20 04:37:00,NaT,NaT,NaT


<function print>

Step 8: % of users who got to checkout but did not purchase

In [10]:
checkout_count = all_data.checkout_time.count()
checkout_count_alternate = all_data[~all_data.checkout_time.isnull()]
print(len(checkout_count_alternate))
purchase_count = all_data.purchase_time.count()

print(checkout_count)

checkout_no_purchase = all_data[(all_data.purchase_time.isnull()) & (~all_data.checkout_time.isnull())]
print(purchase_count)
print(len(checkout_no_purchase))

# What I have written below is a way to get the number of users who made it to checkout but did not purchase
# without using the len() function. It requires chaining more pandas methods together and I think
# is more syntactically complex so probably not a good idea
checkout_no_purchase_method_chain = all_data[(all_data.purchase_time.isnull()) & (~all_data.checkout_time.isnull())]\
.user_id.count()
print(checkout_no_purchase_method_chain)

percent_users_checkout_no_purchase = len(checkout_no_purchase) / checkout_count * 100
print(percent_users_checkout_no_purchase)

334
334
252
82
82
24.550898203592812


Step 9: check each part of the funnel, let's print all 3 of them again

In [11]:
# Calculated in previous part of project. Total number of visitors to cooltshirts.com
print("CoolTShirts.com had " + str(visit_count) + " visitors.")

visit_no_cart_count = len(all_data[(~all_data.visit_time.isnull()) & (all_data.cart_time.isnull())])
print("CoolTShirts.com had " + str(visit_no_cart_count) + " visitors who did not get to the cart.")

percent_lost_after_visit_before_cart = visit_no_cart_count / visit_count * 100
print("Percent lost after visiting but before getting to cart is " + str(percent_lost_after_visit_before_cart) + "%")

cart_count = len(all_data[~all_data.cart_time.isnull()])
print("CoolTShirts.com had " + str(cart_count) + " visitors make it to the cart.")
## S
cart_no_checkout = len(all_data[(~all_data.cart_time.isnull()) & (all_data.checkout_time.isnull())])
print("CoolTShirts.com had " + str(cart_no_checkout) + " visitors make it to the cart but not check out.")

percent_lost_after_cart_before_checkout = round(cart_no_checkout / cart_count * 100, 1)
print("Percent lost after cart but before checkout is " + str(percent_cart_no_checkout) + "%")

checkout_count = len(all_data[~all_data.checkout_time.isnull()])
print("CoolTShirts.com had " + str(checkout_count) + " visitors make it to the checkout.")

checkout_no_purchase = len(all_data[(~all_data.checkout_time.isnull()) & (all_data.purchase_time.isnull())])
print("CoolTShirts.com had " + str(checkout_no_purchase) + " visitors make it to the check out but not purchase.")

percent_lost_after_checkout_before_purhcase = round(checkout_no_purchase / checkout_count * 100, 1)

print("Percent lost after checkout but before purchase " + str(percent_lost_after_checkout_before_purhcase) + "%")

purchase_count = len(all_data[~all_data.purchase_time.isnull()])
print("CoolTShirts.com had " + str(checkout_count) + " visitors make it to the purchase.")

print(len(all_data.cart_time.isnull()))
cart_null_count = len(all_data[all_data.cart_time.isnull()])
print(cart_null_count)
print(1652 + 456)
print(len(all_data))
print(len(visits))
print(len(cart))

CoolTShirts.com had 2000 visitors.
CoolTShirts.com had 1652 visitors who did not get to the cart.
Percent lost after visiting but before getting to cart is 82.6%
CoolTShirts.com had 456 visitors make it to the cart.
CoolTShirts.com had 122 visitors make it to the cart but not check out.
Percent lost after cart but before checkout is 35.1%
CoolTShirts.com had 334 visitors make it to the checkout.
CoolTShirts.com had 82 visitors make it to the check out but not purchase.
Percent lost after checkout but before purchase 24.6%
CoolTShirts.com had 334 visitors make it to the purchase.
2108
1652
2108
2108
2000
348


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [12]:
all_data['time_to_purchase'] = all_data.purchase_time - all_data.visit_time


                                   user_id          visit_time  \
0     943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1     0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2     6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3     6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4     a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
...                                    ...                 ...   
2103  33913ac2-03da-45ae-8fc3-fea39df827c6 2017-03-25 03:29:00   
2104  4f850132-b99d-4623-80e6-6e61d003577e 2017-01-08 09:57:00   
2105  f0830b9b-1f5c-4e74-b63d-3f847cc6ce70 2017-09-07 12:56:00   
2106  b01bffa7-63ba-4cd3-9d93-eb1477c23831 2017-07-20 04:37:00   
2107  0336ca81-8d68-443f-9248-ac0b8ad147d5 2017-11-15 10:11:00   

               cart_time       checkout_time       purchase_time  \
0                    NaT                 NaT                 NaT   
1    2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00   
2  

Step 11: examine the results

In [ ]:
print(all_data)

Step 12: average time to purchase

In [13]:
average_time_to_purchase = all_data.time_to_purchase.mean()
print(average_time_to_purchase)

0 days 00:43:12.380952380
